In [2]:
import numpy as np
from toolbox.load_assets import load_sms_spam
from toolbox.prepare_data import prepare
from toolbox.conversion import conversion
from toolbox.reshape import reshape
from sklearn.neural_network import MLPClassifier

file = "./data/spam.csv"
X = "./data/train.csv"
spam_dic = load_sms_spam(file)
temp = []

data = prepare(X,'sms','label')
msg = data[0]
labels = data[1]


converted_data = conversion(spam_dic, msg)
reshape(converted_data)

mlp = MLPClassifier(solver='adam',hidden_layer_sizes=15,max_iter=3000)
mlp.fit(converted_data,labels)

print(mlp.score(converted_data, labels))



0.9935414424111948
